<a href="https://colab.research.google.com/github/mauricioaoki-ux/Projeto-DIO-Crie-sua-propria-solucao/blob/main/Assistente_de_Voz_B%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# Referência: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=10):
  # Executa o código JavaScript para gravar o áudio
  display(Javascript(RECORD))
  # Recebe o áudio gravado como resultado do JavaScript
  js_result = output.eval_js('record(%s)' % (sec * 1000))
   # Decodifica o áudio em base64
  audio = b64decode(js_result.split(',')[1])
  # Salva o áudio em um arquivo
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  # Retorna o caminho do arquivo de áudio (pasta padrão do Google Colab)
  return f'/content/{file_name}'

# Grava o áudio do usuário por um tempo determinado (padrão 5 segundos)
print('Ouvindo...\n')
record_file = record()

# Exibe o áudio gravado
display(Audio(record_file, autoplay=False))

Ouvindo...



<IPython.core.display.Javascript object>

In [66]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [71]:
import whisper

# Carrega o modelo Whisper. Você pode escolher outros modelos como 'tiny', 'base', 'small', 'medium', 'large'
# 'base' é um bom ponto de partida para a maioria das necessidades.
model = whisper.load_model("small")

# Transcreve o áudio gravado. Use 'record_file' que é o arquivo salvo na primeira célula.
# Ou substitua por 'minha_gravacao_5s.wav' ou 'nova_gravacao.wav' se você os gravou.
# Vamos usar 'record_file' como padrão, que é '/content/request_audio.wav'
# Certifique-se de que este arquivo existe após a gravação.

# Exemplo: usando o arquivo salvo pela função record() na primeira célula
audio_to_transcribe = record_file # '/content/request_audio.wav'

# Ou se você gravou com a função record_audio_from_browser:
# audio_to_transcribe = 'minha_gravacao_5s.wav' # ou 'nova_gravacao.wav', 'minha_gravacao_10s.wav'

print(f"Transcrevendo o arquivo: {audio_to_transcribe}")

try:
  # Faz a transcrição
  result = model.transcribe(audio_to_transcribe)
  print("Transcrição:")
  print(result["text"])
except FileNotFoundError:
  print(f"Erro: O arquivo de áudio '{audio_to_transcribe}' não foi encontrado. Por favor, grave o áudio primeiro.")
except Exception as e:
  print(f"Ocorreu um erro durante a transcrição: {e}")

Transcrevendo o arquivo: /content/request_audio.wav


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcrição:
 to stay on prevention balanced but responsible you


# Nova seção

In [75]:
!pip install gTTS

In [76]:
from gtts import gTTS
from IPython.display import Audio, display

# Pega a transcrição do último resultado
# Certifique-se de que 'result' está definido e contém a transcrição
if 'result' in locals() and 'text' in result:
  transcribed_text = result['text']
  print(f"Convertendo a transcrição para áudio: \n\" {transcribed_text} \"")

  # Cria um objeto gTTS (idioma português, velocidade lenta)
  tts = gTTS(text=transcribed_text, lang='pt', slow=False)

  # Salva o áudio em um arquivo temporário
  audio_output_file = 'transcription_audio.mp3'
  tts.save(audio_output_file)

  print(f"Áudio salvo como '{audio_output_file}'. Reproduzindo...")
  # Reproduz o áudio
  display(Audio(audio_output_file, autoplay=True))
else:
  print("Erro: Nenhuma transcrição encontrada. Por favor, execute a célula de transcrição primeiro.")

Convertendo a transcrição para áudio: 
"  to stay on prevention balanced but responsible you "
Áudio salvo como 'transcription_audio.mp3'. Reproduzindo...
